In [1]:
!python3 -m pip install --upgrade pip
!pip install transformers accelerate sentencepiece bitsandbytes faiss-gpu
!pip install tabulate==0.9.0
!pip install python-dotenv tiktoken
!pip install openai==1.45.0
!pip install transformers accelerate sentencepiece bitsandbytes faiss-gpu
!pip install langdetect==1.0.9
!pip install lingua-language-detector==2.0.2

In [2]:
import csv
import os
from pathlib import Path
import pandas as pd
import numpy as np
from tabulate import tabulate
import faiss

import openai
from openai import OpenAI
import tiktoken
from dotenv import load_dotenv

from langdetect import detect
import ast

In [3]:
import time
wait_time=2

In [4]:
# パスの定義
data_path = Path('../input')
csv_output_dir = data_path / 'embeddings_csv'  # 埋め込みベクトル付きCSVファイルのディレクトリ
query_csv_path = data_path / 'query.csv'  # 質問データのパス

In [5]:
load_dotenv()

True

In [6]:
openai.api_key = os.getenv("OPENAI_API_KEY")

MODEL4o_NAME = "gpt-4o-2024-05-13"
MODEL4o_MINI_NAME = "gpt-4o-mini-2024-07-18"

# MODEL4o_NAME = "o1-preview-2024-09-12"
# MODEL4o_NAME = "o1-mini-2024-09-12"

# モデルとエンコーディングの設定
# EMBEDDING_MODEL = "text-embedding-ada-002"
# EMBEDDING_MODEL = "text-embedding-3-small"
EMBEDDING_MODEL = "text-embedding-3-large"

embedding_encoding = "cl100k_base"
encoding = tiktoken.get_encoding(embedding_encoding)

# max_tokens = 128
# max_tokens = 3072
max_tokens = 200
TEMPERATURE = 0.7

# OpenAIクライアントの初期化
client = OpenAI()

In [7]:
# エンコーディングの取得
encoding = tiktoken.get_encoding(embedding_encoding)

In [8]:
def chunk_text(text, max_tokens, encoding):
    tokens = encoding.encode(text)
    chunks = []
    for i in range(0, len(tokens), max_tokens):
        chunk_tokens = tokens[i:i + max_tokens]
        chunk_text = encoding.decode(chunk_tokens)
        chunks.append(chunk_text)
    return chunks


In [9]:
def get_embedding(text, model=EMBEDDING_MODEL):
    text = text.replace("\n", " ")
    response = client.embeddings.create(input=[text], model=model)
    return response.data[0].embedding

In [10]:
def create_faiss_index_from_csv(csv_dir):
    # dimension = 1536  # 埋め込みベクトルの次元数（コメントアウト）
    index = None  # インデックスを後で初期化
    metadata = []

    for csv_file in sorted(csv_dir.glob('*_embeddings.csv')):
        print(f"Loading embeddings from {csv_file.name}...")
        df = pd.read_csv(csv_file, encoding='utf-8-sig')

        # 埋め込みベクトルをNumPy配列に変換（ast.literal_evalを使用）
        embeddings_list = df['embedding'].apply(ast.literal_eval).values
        embeddings = np.vstack(embeddings_list).astype('float32')

        # 埋め込みベクトルの次元数を取得
        n_vectors, embedding_dim = embeddings.shape
        print(f"Embeddings shape: {embeddings.shape}")

        # インデックスが未作成の場合、適切な次元数で初期化
        if index is None:
            index = faiss.IndexFlatL2(embedding_dim)
        else:
            # 既存のインデックスと次元数が異なる場合はエラーを出力
            assert embedding_dim == index.d, f"Embedding dimension {embedding_dim} does not match index dimension {index.d}"

        # FAISSインデックスにベクトルを追加
        index.add(embeddings)

        # メタデータを保存（CSVファイル名を含む）
        num_entries = len(df)
        csv_file_name = csv_file.name
        metadata.extend(zip(df['title'], df['author'], df['text'], [csv_file_name]*num_entries))

    print(f"Indexed {index.ntotal} vectors.")
    return index, metadata


In [11]:
def create_faiss_index_from(df, csv_file_name):
    # dimension = 1536  # 埋め込みベクトルの次元数（コメントアウト）
    index = None  # インデックスを後で初期化
    metadata = []

    print(f"Loading embeddings from {csv_file_name}...")

    # 埋め込みベクトルをNumPy配列に変換
    embeddings_list = df['embedding'].values
    embeddings = np.vstack(embeddings_list).astype('float32')

    # 埋め込みベクトルの次元数を取得
    n_vectors, embedding_dim = embeddings.shape
    print(f"Embeddings shape: {embeddings.shape}")

    # インデックスが未作成の場合、適切な次元数で初期化
    if index is None:
        index = faiss.IndexFlatL2(embedding_dim)
    else:
        # 既存のインデックスと次元数が異なる場合はエラーを出力
        assert embedding_dim == index.d, f"Embedding dimension {embedding_dim} does not match index dimension {index.d}"

    # FAISSインデックスにベクトルを追加
    index.add(embeddings)

    # メタデータを保存（CSVファイル名を含む）
    num_entries = len(df)
    metadata.extend(zip(df['title'], df['author'], df['text'], [csv_file_name]*num_entries))

    return index, metadata


In [12]:
def generate_search_queries(model_name, text, count):
    # テキストから検索クエリを生成する
    
    prompt = [
        {"role": "system", "content": "You want to answer the question using search . What do you type in the search box ?"},
        {"role": "system", "content": f"Please formulate {count} distinct search queries based on the content of the Input text."},
        # {"role": "system", "content": "Please ensure that the output is in English."},
        {"role": "system", "content": "Write the queries you will use in the following"},
        {"role": "system", "content": "format :\n query 1\n query 2\n..."},
        {"role": "user", "content": f"Input text: {text}"},
        {"role": "user", "content": "format :\n"}
    ]
    
    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        temperature=TEMPERATURE,
    )
    
    search_queries = response.choices[0].message.content
    return search_queries

In [13]:
# クエリテキストで検索を行う関数
def search_faiss(index, query_text, metadata, k=3, max_tokens=3072):
    
    # クエリテキストをチャンクに分割
    query_chunks = chunk_text(query_text, max_tokens, encoding)

    # クエリテキストを埋め込みベクトルに変換
    query_embeddings = np.vstack([get_embedding(chunk) for chunk in query_chunks]).astype('float32')

    # FAISSインデックスで検索を実行
    D, I = index.search(query_embeddings, k)
    index.search(query_embeddings, k-2)

    # 重複なしのタイトルと作者を収集
    unique_titles_authors_files = set()

    # 検索結果の表示
    search_text = ""
    for i, idx in enumerate(I[0]):
        title, author, text, csv_file_name = metadata[idx]
        unique_titles_authors_files.add((title, author, csv_file_name))  # 重複を避けるためセットに追加
        search_text += "\n"+text
        # print(f"Result {i+1}:")
        # print(f"Title: {title}")
        # print(f"Author: {author}")
        # print(f"Text: {text[:200]}...")  # テキストの最初の200文字を表示
        # print(f"Distance: {D[0][i]}")
        # print('---')

    # ユニークなタイトルと作者の一覧を表示
    print("Unique Titles and Authors:")
    for title, author, csv_file_name in unique_titles_authors_files:
        print(f"Title: {title}, Author: {author}, CSV File: {csv_file_name}")

    # 対応するCSVファイルを取得
    csv_files_needed = set([csv_file_name for _, _, csv_file_name in unique_titles_authors_files])
    
    return csv_files_needed, search_text


In [14]:
# query.csvの読み込み
query_df = pd.read_csv(query_csv_path)
# query_df[:1]
query_df.head()

,index,problem
0,1,競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？
1,2,骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。
2,3,骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？
3,4,殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？
4,5,小説「芽生」で出てくる国名は何種類ですか？


In [15]:
# FAISSインデックスの作成
index, metadata = create_faiss_index_from_csv(csv_output_dir)

Loading embeddings from 1_embeddings.csv...
Embeddings shape: (6, 3072)
Loading embeddings from 2_embeddings.csv...
Embeddings shape: (49, 3072)
Loading embeddings from 3_embeddings.csv...
Embeddings shape: (13, 3072)
Loading embeddings from 4_embeddings.csv...
Embeddings shape: (7, 3072)
Loading embeddings from 5_embeddings.csv...
Embeddings shape: (10, 3072)
Loading embeddings from 6_embeddings.csv...
Embeddings shape: (26, 3072)
Loading embeddings from 7_embeddings.csv...
Embeddings shape: (5, 3072)
Indexed 116 vectors.


In [16]:
def count_tokens(text, model_name=MODEL4o_NAME):
    tokenizer = tiktoken.encoding_for_model(model_name)
    tokens = tokenizer.encode(text)
    return len(tokens)

In [17]:
def translate_to_japanese(detailed_outline, model_name=MODEL4o_MINI_NAME):
    """
    Translates the detailed outline of a Wikipedia page from English to Japanese.
    
    Parameters:
    model_name (str): The model to be used for translation.
    detailed_outline (str): The detailed outline in English.
    
    Returns:
    str: The translated outline in Japanese.
    """
    prompt = [
        {"role": "system", "content": "You need to translate the following English text into Japanese."},
        {"role": "user", "content": detailed_outline},
        {"role": "system", "content": "Please provide the translation of the entire text into Japanese, maintaining the accuracy and context of the original information."},
        {"role": "system", "content": "必ず日本語に翻訳してください。ローマ字も日本語に翻訳してください"},
        {"role": "system", "content": "翻訳した結果のみを生成してください。"},
    ]
    
    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=TEMPERATURE,
    )
    
    translated_outline = response.choices[0].message.content
    
    return translated_outline

In [18]:
# 質問に対する回答を生成する関数
def generate_answer(query, context_texts, model_name=MODEL4o_NAME):
    context = context_texts

    # 回答に関する注意点
    instructions = [
        "Derive the answer from the provided novel excerpt.",
        "Include units when answering questions that require numerical values.",
        "If you cannot find clues to the answer in the provided text, respond with 'I don't know'.",
        "If you determine that the question itself is incorrect, respond with 'Wrong question'.",
        "Provided novel excerptの内容に答えのヒントが見つからない場合は、 わからない と答えなさい。",
        "問題そのものが間違っていると判断した場合は、 間違えている。 と答えなさい。",
        "Answer the question precisely without any excess or deficiency.",
        "Each answer has an upper limit of tokens; do not exceed it. The current limit is 50 tokens.",
        "Although the rationale for the answer is not evaluated in this competition, it is intended to be used for evaluation on a different axis after the competition. Therefore, please provide accurate information instead of incorrect or fabricated content.",
    ]

    # プロンプトの作成
    prompt = []

    # システムメッセージに注意点を一行ずつ追加
    for instruction in instructions:
        prompt.append({"role": "system", "content": instruction})

    # コンテキストをシステムメッセージとして追加
    prompt.append({"role": "system", "content": f"Provided novel excerpt:\n{context}"})

    # ユーザーメッセージに質問を追加
    prompt.append({"role": "user", "content": f"Question: {query}\nAnswer:"})

    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        # response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    result_str = response.choices[0].message.content

    return result_str
    # print(result_str)
    
    # # JSON形式の文字列を辞書に変換
    # summary_method_name = json.loads(result_str)
    
    # 出力と新しいメッセージをステートに反映
    # return summary_method_name["extract_summary"]

In [19]:
def generate_evidence(translated_result, question, context_texts, model_name=MODEL4o_NAME):
    # 根拠生成に関わる内容のリスト化
    instructions = [
        "Your task is to provide an excerpt from the provided text that serves as evidence for the given answer to the question.",
        "Extract the exact portion from the provided text that supports the answer.",
        "The excerpt should be accurate and taken directly from the text without any modifications.",
        "If you cannot find any relevant information in the provided text, respond with 'なし' (none)."
    ]

    # プロンプトの作成
    prompt = []

    # システムメッセージに注意点を一行ずつ追加
    for instruction in instructions:
        prompt.append({"role": "system", "content": instruction})

    # コンテキストをシステムメッセージとして追加
    prompt.append({"role": "system", "content": f"Question:\n{question}"})
    prompt.append({"role": "system", "content": f"Answer:\n{translated_result}"})
    prompt.append({"role": "system", "content": f"Provided text:\n{context_texts}"})

    # ユーザーメッセージに質問と回答を追加
    prompt.append({"role": "user", "content": f"Please explain the rationale for the Answer using the Provided text for the Question."})
    prompt.append({"role": "user", "content": f"Please answer only the rationale."})

    response = client.chat.completions.create(
        model=model_name,
        messages=prompt,
        temperature=TEMPERATURE,
    )
    evidence = response.choices[0].message.content.strip()

    # 根拠が空の場合や「なし」と判断される場合
    if evidence in ["なし", "わかりません", "分かりません", "I don't know", "Wrong question", "None", "No relevant information found"]:
        evidence = "なし"
    return evidence


In [20]:
def sanitize_text(text):
    # 改行をスペースに置換し、カンマを全角カンマに置換
    return text.replace('\n', ' ').replace('\r', ' ').replace(',', '、')

In [21]:
%%time
# predictions.csvを保存するリスト
predictions = []
# 各質問に対して処理を行う
for idx, row in query_df.iterrows():
    question_index = row['index']
    question = row['problem']
    # print(f"Processing question {question_index}: {question}")
    
    print(f"index: {idx}")
    
    # FAISSで検索して関連するテキストを取得
    csv_files_needed, search_text = search_faiss(index, question, metadata, k=5)
    time.sleep(wait_time)
    
    # 必要なCSVファイルを読み込み、テキストデータを収集
    context_texts = ''
    for csv_file_name in csv_files_needed:
        csv_file_path = csv_output_dir / csv_file_name
        df = pd.read_csv(csv_file_path, encoding='utf-8-sig')
        print(f"\nProcessing CSV File: {csv_file_name}")
        # データフレームのテキスト列を一つの文字列に結合
        title_text = df['title'].iloc[0]  # 修正：iloc[0] で最初の要素を取得
        author_text = df['author'].iloc[0]
        text = ' '.join(df['text'].tolist())
        # textに対して再検索
        # テキストをチャンクに分割
        text_chunks = chunk_text(text, max_tokens, encoding)

        # 各チャンクに対して埋め込みを生成し、データフレームに保存
        data = []
        for chunk in text_chunks:
            embedding = get_embedding(chunk)
            data.append({'text': chunk, 'embedding': embedding, 'title': title_text, 'author': author_text})
        df = pd.DataFrame(data, columns=['text', 'embedding', 'title', 'author'])
        
        # FAISSインデックスの作成
        re_index, re_metadata = create_faiss_index_from(df, csv_file_name)
        csv_files, re_search_text = search_faiss(re_index, question, re_metadata, k=512)
        time.sleep(wait_time)
        
        # context_textsに検索対象になったテキストを追加
        # context_texts += f"タイトル: {title_text}\n作者: {author_text}\n{text}\n\n"
        context_texts += re_search_text

    context_token_num = count_tokens(question + context_texts)
    if context_token_num > 128000:
        csv_files, re_search_text = search_faiss(re_index, question, re_metadata, k=256)
        time.sleep(wait_time)
        # 再検索結果を前の検索に追加する
        search_query = generate_search_queries(MODEL4o_NAME, question, "three")
        time.sleep(wait_time)
        re_csv_files_needed, re__search_text = search_faiss(re_index, search_query, re_metadata, k=256)
        # トークン数が使用できる量より多い場合、使用するテキストは検索結果のテキスト
        context_texts = re_search_text + re__search_text
        time.sleep(wait_time)

    # 回答を生成
    result_str = generate_answer(question, context_texts, model_name=MODEL4o_NAME)
    time.sleep(wait_time)
    translated_result = result_str
    # トークン量計算
    token_num = count_tokens(result_str)
    while token_num > 50:
        # print("回答が50トークンを超えています。再生成します。")
        result_str = generate_answer(question, context_texts, model_name=MODEL4o_NAME)
        time.sleep(wait_time)
        # 日本語であることの判定と翻訳
        lang = detect(result_str)
        if lang != "ja":
            # print("Answer is not in Japanese. Translating...")
            translated_result = translate_to_japanese(result_str)
            time.sleep(wait_time)
        else:
            translated_result = result_str

        token_num = count_tokens(translated_result)

    # 根拠を生成
    evidence = generate_evidence(translated_result, question, context_texts, model_name=MODEL4o_NAME)
    time.sleep(wait_time)

    # 日本語であることの判定と翻訳
    lang = detect(evidence)
    if lang != "ja":
        # print("Evidence is not in Japanese. Translating...")
        translated_evidence = translate_to_japanese(evidence)
        time.sleep(wait_time)
    else:
        translated_evidence = evidence

    # 回答と根拠のテキストをサニタイズ
    translated_result = sanitize_text(translated_result)
    translated_evidence = sanitize_text(translated_evidence)
    
    # デバッグ用出力
    print(f"Question: {question}")
    print(f"Answer: {translated_result}")
    print(f"Evidence: {translated_evidence}")

    # predictionsに追加
    predictions.append([question_index, translated_result, translated_evidence])


index: 0
Unique Titles and Authors:
Title: 競漕, Author: 久米正雄, CSV File: 4_embeddings.csv

Processing CSV File: 4_embeddings.csv
Loading embeddings from 4_embeddings.csv...
Embeddings shape: (93, 3072)
Unique Titles and Authors:
Title: 競漕, Author: 久米正雄, CSV File: 4_embeddings.csv
Question: 競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？
Answer: わからない
Evidence: 提供されたテキストには、レガッタの3日前における農業部門と文学部門のレース間の正確な時間差（秒数）に関する具体的な情報が含まれていません。したがって、「わからない」という答えが適切です。
index: 1
Unique Titles and Authors:
Title: サーカスの怪人, Author: 江戸川乱歩, CSV File: 6_embeddings.csv

Processing CSV File: 6_embeddings.csv
Loading embeddings from 6_embeddings.csv...
Embeddings shape: (392, 3072)
Unique Titles and Authors:
Title: サーカスの怪人, Author: 江戸川乱歩, CSV File: 6_embeddings.csv
Question: 骸骨男の正体は誰ですか？作中で言及されている氏名で答えること。
Answer: 遠藤平吉
Evidence: 遠藤平吉が骸骨男の真の正体であると特定する理由は、テキストがこの情報を明示的に示しているからです。以下がその支持する抜粋です：  「私は、きのう、明智さんに、いろいろ話を聞いているうちに、やっと、思いだした。わしを、こんなひどいめにあわせるやつは、遠藤平吉のほかにはない。」
index: 2
Unique Titles and Authors:
Title: サーカスの怪人, Auth

In [22]:
# DataFrameのカラム名を指定
predictions_df = pd.DataFrame(predictions, columns=['QuestionIndex', 'Answer', 'Evidence'])
predictions_df.head()

,QuestionIndex,Answer,Evidence
0,1,わからない,提供されたテキストには、レガッタの3日前における農業部門と文学部門のレース間の正確な時間差（...
1,2,遠藤平吉,遠藤平吉が骸骨男の真の正体であると特定する理由は、テキストがこの情報を明示的に示しているから...
2,3,わからない,「わからない」という答えの根拠は、提供されたテキストが骸骨男が足跡を残さずにバスからどのよう...
3,4,わからない,なし
4,5,わからない,提供されたテキストには、小説「芽生」に言及されている国の数に関する情報は含まれていません。し...


わからないと回答した結果に対して

In [23]:
def re_generate_answer(question, previous_answer, previous_evidence, part_context_text, model_name=MODEL4o_NAME):
    context = context_texts

    # 回答に関する注意点
    instructions = [
        "Derive the answer from the provided novel excerpt.",
        f"前回は同じ質問に対して {previous_answer} と回答しています。この時の根拠は {previous_evidence} でした。前回の回答も参考にもう一度考えてください。",
        "Include units when answering questions that require numerical values.",
        "If you cannot find clues to the answer in the provided text, respond with 'I don't know'.",
        "If you determine that the question itself is incorrect, respond with 'Wrong question'.",
        "Provided novel excerptの内容に答えのヒントが見つからない場合は、 わからない と答えなさい。",
        "問題そのものが間違っていると判断した場合は、 間違えている。 と答えなさい。",
        "Answer the question precisely without any excess or deficiency.",
        "Each answer has an upper limit of tokens; do not exceed it. The current limit is 50 tokens.",
        "Although the rationale for the answer is not evaluated in this competition, it is intended to be used for evaluation on a different axis after the competition. Therefore, please provide accurate information instead of incorrect or fabricated content.",
    ]

    # プロンプトの作成
    prompt = []

    # システムメッセージに注意点を一行ずつ追加
    for instruction in instructions:
        prompt.append({"role": "system", "content": instruction})

    # コンテキストをシステムメッセージとして追加
    prompt.append({"role": "system", "content": f"Provided novel excerpt:\n{part_context_text}"})

    # ユーザーメッセージに質問を追加
    prompt.append({"role": "user", "content": f"Question: {question}\nAnswer:"})

    # 概要と提案手法名抽出用のプロンプトテンプレートを作成
    response = client.chat.completions.create(
        model=model_name, # model = "deployment_name".
        messages=prompt,
        # response_format={ "type": "json_object" },
        temperature=TEMPERATURE,
    )
    result_str = response.choices[0].message.content

    return result_str


In [24]:
# 'Answer' 列が「わからない」の行のインデックスを取得
unknown_answers = predictions_df[predictions_df["Answer"] == 'わからない'].index.tolist()
unknown_answers += predictions_df[predictions_df["Answer"] == "I don't know"].index.tolist()
unknown_answers += predictions_df[predictions_df["Answer"] == "I don't know."].index.tolist()
unknown_answers += predictions_df[predictions_df["Answer"] == "なし"].index.tolist()
print(f"'わからない' の回答が含まれる行のインデックス: {unknown_answers}")

'わからない' の回答が含まれる行のインデックス: [0, 2, 3, 4, 6, 9, 11, 12, 13, 15, 16, 18, 19, 20, 24, 28, 29, 30, 31, 32, 34, 35, 36, 39, 42, 44, 46, 47, 49, 50, 51, 53, 54, 55]


In [25]:
for idx in unknown_answers:
    # 質問番号を取得
    question_index = predictions_df.loc[idx, 'QuestionIndex']  # カラム0がQuestionIndex
    # 対応する質問を query_df から取得
    question_row = query_df[query_df['index'] == question_index].iloc[0]
    question = question_row['problem']
    print(f"再処理: Question Index {question_index}, Question: {question}")
    print(f"Answer: {predictions_df.loc[idx, 'Answer']}\nEvidence: {predictions_df.loc[idx, 'Evidence']}")
    # 元の根拠を取得
    previous_answer = predictions_df.loc[idx, 'Answer']
    previous_evidence = predictions_df.loc[idx, 'Evidence']
    # new_answer = re_generate_answer(question, previous_answer, previous_evidence, model_name=MODEL4o_NAME)

    # FAISSで検索して関連するテキストを取得（k=3）
    csv_files_needed, search_text = search_faiss(index, question, metadata, k=3)

    # 各CSVファイルに対して処理を行う
    new_answer = 'わからない'  # 初期値として「わからない」を設定
    token_num = count_tokens(new_answer)
    translated_new_answer = ""
    translated_evidence = ""
    for csv_file_name in csv_files_needed:
        csv_file_path = csv_output_dir / csv_file_name
        df = pd.read_csv(csv_file_path, encoding='utf-8-sig')

        # タイトル、作者、全文を取得
        title_text = df['title'].iloc[0]
        author_text = df['author'].iloc[0]
        full_text = ' '.join(df['text'].tolist())
        print(title_text, author_text)
        # context_textsを構築
        title_author_text = f"タイトル: {title_text}\n作者: {author_text}\n"
        full_context_text = title_author_text + full_text + '\n\n'

        # 質問と前回の回答、根拠を結合してトークン数を計算
        tokens_used_by_question_and_previous = count_tokens(question + previous_answer + previous_evidence)
        max_total_tokens = 128000
        max_context_tokens = max_total_tokens - tokens_used_by_question_and_previous

        if max_context_tokens <= 0:
            # トークン数が不足している場合
            print("トークン数が不足しているため、コンテキストを分割して処理します。")

        # context_textsのトークン数を計算
        context_token_num = count_tokens(full_context_text)
        
        # トークン数が超過する場合はテキストを分割
        if context_token_num > max_total_tokens:
            print("context_textsがトークン制限を超えているため、分割して処理します。")
            # トークン数に基づいて何分割するか計算
            num_splits = int(np.ceil(context_token_num / max_context_tokens))
            split_size = int(len(full_text) / num_splits)  # テキストを文字数で分割

            for i in range(num_splits):
                start_idx = i * split_size
                end_idx = (i + 1) * split_size if i < num_splits - 1 else len(full_text)
                part_text = full_text[start_idx:end_idx]
                part_context_text = title_author_text + part_text + '\n\n'

                # 再度トークン数を確認してから再回答を生成
                part_context_token_num = count_tokens(part_context_text + question + previous_answer + previous_evidence)
                
                if part_context_token_num <= max_total_tokens:
                    # re_generate_answer関数を呼び出して回答を生成
                    new_answer = re_generate_answer(question, previous_answer, previous_evidence, part_context_text, model_name=MODEL4o_NAME)
                    translated_new_answer = new_answer
                    # トークン量計算
                    token_num = count_tokens(new_answer)
                    while token_num > 50:
                        # print("回答が50トークンを超えています。再生成します。")
                        new_answer = re_generate_answer(question, previous_answer, previous_evidence, part_context_text, model_name=MODEL4o_NAME)

                        # 日本語であることの判定と翻訳
                        lang = detect(new_answer)
                        if lang != "ja":
                            # print("Answer is not in Japanese. Translating...")
                            translated_new_answer = translate_to_japanese(new_answer)
                        else:
                            translated_new_answer = new_answer
                        
                        token_num = count_tokens(translated_new_answer)

                    # 根拠を生成
                    evidence = generate_evidence(translated_new_answer, question, part_context_text, model_name=MODEL4o_NAME)

                    # 日本語であることの判定と翻訳
                    lang = detect(evidence)
                    if lang != "ja":
                        # print("Evidence is not in Japanese. Translating...")
                        translated_evidence = translate_to_japanese(evidence)
                    else:
                        translated_evidence = evidence

                    # 回答が「わからない」でない場合、ループを抜ける
                    if new_answer != 'わからない' or new_answer != 'なし':
                        break
                else:
                    print(f"分割後のcontext_textsでもトークン数が超過しています。部分をさらに分割します。")

            # 分割処理後、回答が得られた場合、外側のループを抜ける
            if translated_new_answer != 'わからない' or new_answer != 'なし':
                break
        else:
            # context_textsがトークン制限内の場合
            new_answer = re_generate_answer(question, previous_answer, previous_evidence, full_context_text, model_name=MODEL4o_NAME)
            translated_new_answer = new_answer
            # トークン量計算
            token_num = count_tokens(new_answer)
            while token_num > 50:
                # print("回答が50トークンを超えています。再生成します。")
                new_answer = re_generate_answer(question, previous_answer, previous_evidence, full_context_text, model_name=MODEL4o_NAME)

                # 日本語であることの判定と翻訳
                lang = detect(new_answer)
                if lang != "ja":
                    # print("Answer is not in Japanese. Translating...")
                    translated_new_answer = translate_to_japanese(new_answer)
                else:
                    translated_new_answer = new_answer
                token_num = count_tokens(translated_new_answer)
                
            
            # 根拠を生成
            evidence = generate_evidence(translated_new_answer, question, full_context_text, model_name=MODEL4o_NAME)

            # 日本語であることの判定と翻訳
            lang = detect(evidence)
            if lang != "ja":
                # print("Evidence is not in Japanese. Translating...")
                translated_evidence = translate_to_japanese(evidence)
            else:
                translated_evidence = evidence

    # 回答と根拠のテキストをサニタイズ
    translated_new_answer = sanitize_text(translated_new_answer)
    if translated_new_answer == "なし":
        translated_new_answer = "わからない"
    elif translated_new_answer == "I don't know.":
        translated_new_answer = "わからない"
    elif translated_new_answer == "I don't know":
        translated_new_answer = "わからない"

    translated_evidence = sanitize_text(translated_evidence)
    # 最終的な回答をデータフレームに更新
    predictions_df.at[idx, 'Answer'] = translated_new_answer
    predictions_df.at[idx, 'Evidence'] = translated_evidence
    print("再回答の結果")
    print(f"Answer: {predictions_df.loc[idx, 'Answer']}\nEvidence: {predictions_df.loc[idx, 'Evidence']}")
    print(token_num)


再処理: Question Index 1, Question: 競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？
Answer: わからない
Evidence: 提供されたテキストには、レガッタの3日前における農業部門と文学部門のレース間の正確な時間差（秒数）に関する具体的な情報が含まれていません。したがって、「わからない」という答えが適切です。
Unique Titles and Authors:
Title: 競漕, Author: 久米正雄, CSV File: 4_embeddings.csv
競漕 久米正雄
再回答の結果
Answer: わからない
Evidence: 本文には「競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？」に関する具体的な秒数の情報が記載されていないため、答えを導き出すことができません。
3
再処理: Question Index 3, Question: 骸骨男はバスの中に足跡を一切残さずにどうやって抜け出しましたか？
Answer: わからない
Evidence: 「わからない」という答えの根拠は、提供されたテキストが骸骨男が足跡を残さずにバスからどのように逃げたのかを明示的に述べていないことです。テキストでは骸骨男に関するさまざまな神秘的な行動や逃走シナリオが描かれていますが、この特定の逃走に関する明確な説明やメカニズムは提供されていません。したがって、与えられたテキストに基づくと、答えは「わからない」ということになります。
Unique Titles and Authors:
Title: サーカスの怪人, Author: 江戸川乱歩, CSV File: 6_embeddings.csv
サーカスの怪人 江戸川乱歩
再回答の結果
Answer: わからない
Evidence: 骸骨男が何者か判明していないため、どのようにして足跡を残さずに抜け出したかはわからない。
3
再処理: Question Index 4, Question: 殺人罪で裁判にかけられた兄が登場しますが、その理由は何ですか？
Answer: わからない
Evidence: なし
Unique Titles and Authors:
Title: サーカスの怪人, Author: 江戸川

In [26]:
# predictions.csvとして保存（ヘッダーなし）
predictions_csv_path = 'predictions.csv'
predictions_df.to_csv(
    predictions_csv_path,
    index=False,
    header=False,
    encoding='utf-8-sig',
    quoting=csv.QUOTE_MINIMAL  # 必要に応じて引用符を使用
)
print(f"Saved predictions to {predictions_csv_path}")

Saved predictions to predictions.csv


In [27]:
# CSVファイルを再読み込みして確認
test_df = pd.read_csv(predictions_csv_path, header=None)
print("Test DataFrame:")
print(test_df.head())

Test DataFrame:
   0      1                                                  2
0  1  わからない  本文には「競漕会の三日前のレースコースでの結果は、農科と文科でどれくらいの秒数差があったか？...
1  2   遠藤平吉  遠藤平吉が骸骨男の真の正体であると特定する理由は、テキストがこの情報を明示的に示しているから...
2  3  わからない       骸骨男が何者か判明していないため、どのようにして足跡を残さずに抜け出したかはわからない。
3  4  わからない  質問にある「殺人罪で裁判にかけられた兄」について、提供されたテキストには具体的な説明が欠けて...
4  5  わからない                                                 なし


In [28]:
# 応募用ファイルを作成（zipに圧縮）
import zipfile
submission_zip_path = 'submit.zip'
with zipfile.ZipFile(submission_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipf.write(predictions_csv_path)
print(f"Created submission file: {submission_zip_path}")

Created submission file: submit.zip
